In [1]:
# imports
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import sys
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.preprocessing import LabelEncoder
# local
import acquire as a
import t_prepare as p
import modeling as m

In [2]:
df = a.get_wine()

In [3]:
dfr = p.prep_wine(df)
dfr.head()
dfr = dfr.drop(columns='bound_sulfur_dioxide')

In [4]:
dfr = dfr[dfr['red'] == 1]

In [5]:
#
train, validate, test = p.split_data(dfr)

In [6]:
train.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,red
1170,9.2,0.360,0.34,1.6,0.062,5.0,12.0,0.99667,3.20,0.67,10.5,6,1
1501,7.8,0.820,0.29,4.3,0.083,21.0,64.0,0.99642,3.16,0.53,9.4,5,1
1458,8.4,0.290,0.40,1.7,0.067,8.0,20.0,0.99603,3.39,0.60,10.5,5,1
286,12.0,0.450,0.55,2.0,0.073,25.0,49.0,0.99970,3.10,0.76,10.3,6,1
1178,5.6,0.915,0.00,2.1,0.041,17.0,78.0,0.99346,3.68,0.73,11.4,5,1


In [7]:
Xtr,Xv,Xt = train.drop(columns=['quality']),validate.drop(columns=['quality']),test.drop(columns=['quality'])
Xtr_s,Xv_s,Xt_s = p.std(Xtr,Xv,Xt)
ytr,yv,yt = train[['quality']],validate[['quality']],test[['quality']]
Xtr_s.sample(1)

,fixed_acidity_s,volatile_acidity_s,citric_acid_s,residual_sugar_s,chlorides_s,free_sulfur_dioxide_s,total_sulfur_dioxide_s,density_s,pH_s,sulphates_s,alcohol_s,red_s
1560,-0.262672,0.380422,-0.007117,-0.398011,-0.058874,1.425153,2.552756,-0.239753,-0.71521,-0.835457,-0.550704,0.0


In [8]:

metric = m.reg_mods(Xtr,ytr,Xv,yv,alpha=[1,2],degree=[2,3],features=None)


/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_glm/glm.py:294: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res)
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_glm/glm.py:294: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res)
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_glm/glm.py:294: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iter

In [9]:
metric

,model,features,params,rmse_tr,rmse_v,r2_tr,r2_v
0,bl_mean,None,None,0.829874,0.755867,0.0000,-0.00586
1,LinearRegression,[fixed_acidity],None,0.820000,0.750000,0.0172,0.00830
2,LinearRegression,[volatile_acidity],None,0.760000,0.700000,0.1657,0.13220
3,LinearRegression,[citric_acid],None,0.800000,0.740000,0.0689,0.04280
4,LinearRegression,[residual_sugar],None,0.830000,0.760000,0.0000,-0.00710
...,...,...,...,...,...,...,...
20471,LinearRegression,"[fixed_acidity, volatile_acidity, citric_acid,...",None,0.670000,0.590000,0.3573,0.37860
20472,PolynomialFeature,"[fixed_acidity, volatile_acidity, citric_acid,...",degree=2,0.610000,0.610000,0.4561,0.34730
20473,PolynomialFeature,"[fixed_acidity, volatile_acidity, citric_acid,...",degree=3,0.440000,1.070000,0.7242,-1.03090
20474,TweedieRegressor,"[fixed_acidity, volatile_acidity, citric_acid,...","power=0,alpha=1",0.740000,0.660000,0.2070,0.23810


In [10]:

metric = metric.sort_values(['rmse_v', 'r2_v'], ascending = [True, False])


In [11]:
pd.set_option('max_colwidth', None)
metric.groupby('model')[['features','params','rmse_v', 'r2_v']].agg('first')


,features,params,rmse_v,r2_v
model,,,,
LinearRegression,"[fixed_acidity, volatile_acidity, residual_sugar, chlorides, total_sulfur_dioxide, density, sulphates, alcohol]",None,0.590000,0.38780
PolynomialFeature,"[volatile_acidity, total_sulfur_dioxide, sulphates, alcohol]",degree=3,0.580000,0.41660
TweedieRegressor,"[fixed_acidity, volatile_acidity, citric_acid, free_sulfur_dioxide, total_sulfur_dioxide, pH, sulphates, alcohol]","power=0,alpha=1",0.660000,0.23940
bl_mean,None,None,0.755867,-0.00586


In [12]:
pd.set_option('display.max_columns', None)

In [13]:
pd.set_option('max_colwidth', None)
metric.groupby('model')[['features']].agg('first')


,features
model,
LinearRegression,"[fixed_acidity, volatile_acidity, residual_sugar, chlorides, total_sulfur_dioxide, density, sulphates, alcohol]"
PolynomialFeature,"[volatile_acidity, total_sulfur_dioxide, sulphates, alcohol]"
TweedieRegressor,"[fixed_acidity, volatile_acidity, citric_acid, free_sulfur_dioxide, total_sulfur_dioxide, pH, sulphates, alcohol]"
bl_mean,None


#### for Linear Regression, for Polynomial

fixed_acidity | Fixed acidity

volatile_acidity | volatile acidity

null                 | citric acid

residual_sugar | residual sugar

chlorides | null

free_sulfur_dioxide | free_sulfur_dioxide

bound_sulfur_dioxide | null

total_sulfur_dioxide | null 

density | density

pH | pH

sulphates | sulphates

alcohol | null

Next, identify visualizations you want to include